# Approximation of electrostatics matrix by MCBH method

Setting of the problem: for given particles $X_1, \ldots, X_N$ calculate Coulomb potential $f(x, y) = \vert x-y \vert^{-1}$ they create on themselves.

Since goal of this example is to show how to use ***h2tools*** package, we only approximate following matrix:


$$
A_{ij} = \left\{ \begin{array}{cl} \frac{1}{\vert X_i-X_j\vert}, & i \ne j \\ 0, & i = j \end{array} \right.
$$

At first, we import all necessary modules, generate data and set variables.
Then we build cluster trees, approximate matrix with MCBH algorithm and measure relative error of far field approximation.

In [1]:
# main imports and variables

# make this script work with both Python 2 and Python 3
from __future__ import print_function, absolute_import, division

# import all necessary modules and set variables
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
# at first, import predefined functions and dataclass for particles from h2py.collections
from h2tools.collections import particles
# import main container for problem and cluster tree
from h2tools import Problem, ClusterTree

import numpy as np

# set dimensionality of the problem (ndim), size of the problem (count),
# desired relative accuracy of far field approximation (tau),
# number of iterations for MCBH algorithm (iters),
# whether to hold special submatrices in memory or compute them on demand (onfly),
# symmetricity of the problem (symmetric),
# maximum size of leaf node of cluster trees (block_size) and
# verbosity (verbose)
ndim = 3
count = 5000
tau = 1e-5
iters = 1
onfly = 0
symmetric = 1
block_size = 20
verbose = 1
random_init = 2

func = particles.inv_distance

In [2]:
#generate data
# set random seed so that results are repeatable
np.random.seed(0)
# generate positions of particles in shape (ndim, count)
position = np.random.randn(ndim, count)
# create data object from given positions of particles
data = particles.Particles(ndim, count, position)
# initialize cluster tree from data object
tree = ClusterTree(data, block_size)
# create main problem object with interaction particles.inv_distance
problem = Problem(func, tree, tree, symmetric, verbose, None)

Cluster trees are generated in 0.370278835297 seconds
Depth level of each cluster tree: 11
Row cluster tree
    nodes : 719
    leaves: 360
Column cluster tree
    nodes : 719
    leaves: 360


In [3]:
# import multicharge approximation method
from h2tools.mcbh import mcbh

# build approximation of the matrix with MCBH algorithm

matrix = mcbh(problem, tau, iters=iters, onfly=onfly, verbose=verbose, random_init=random_init, mpi_comm=None)

# check approximation error
print(matrix.diffnorm(far_only=1))

# Compress matrix
matrix.svdcompress(1e-4, verbose=1)
print(matrix.diffnorm(far_only=1))

matrix.svdcompress(1e-3, verbose=1)
print(matrix.diffnorm(far_only=1))

Far-field interactions(MCBH method):
    Function calls: 15926
    Function values computed: 45698857
    Function time, seconds: 0.63
    Average time per function value, seconds: 1.38e-08
    Maxvol time, seconds: 6.0511329174
Near-field interactions:
    Function calls: 5756
    Function values computed: 1598473
    Function time, seconds: 0.08
    Average time per function value, seconds: 5.13e-08
Total time, seconds: 7.05
Memory:
    Basises, MB: 0.17
    Transfer matrices, MB: 2.95
    Far-field interactions, MB: 43.02
    Near-field interactions, MB: 13.54
Total memory, MB: 59.68
3.03515027791e-06
memory BEFORE SVD-compression: 59.68MB
memory AFTER SVD-compression: 19.13MB
recompression time: 3.67317700386
2.19236086441e-05
memory BEFORE SVD-compression: 19.13MB
memory AFTER SVD-compression: 18.02MB
recompression time: 3.65114378929
0.000413486690841


In [4]:
print(matrix)

H2matrix at 0x1078ba690
    Structure (h2/mcbh): h2
    Memory layout (low/full): full
    Shape: [5000, 5000]
    Total memory, MB: 18.02
        Basises, MB: 0.01
        Transfer matrices, MB: 1.37
        Far-field interactions, MB: 3.10
        Near-field interactions, MB: 13.54
